In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [4]:
from pathlib import Path

from wikipedia_cleanup.data_filter import KeepAttributesDataFilter, generate_default_filters
from wikipedia_cleanup.predict import TrainAndPredictFramework
from wikipedia_cleanup.predictor import ZeroPredictor, OnePredictor, MeanPredictor, RandomPredictor
from wikipedia_cleanup.property_correlation import PropertyCorrelationPredictor
from wikipedia_cleanup.random_forest import RandomForestPredictor
from datetime import datetime

import pandas as pd
from tqdm import tqdm

In [5]:
model = RandomForestPredictor(use_cache=False)
framework = TrainAndPredictFramework(model, group_key=['infobox_key', 'property_name'],test_start_date=datetime(2017, 9, 1))

In [6]:
import os
import glob
csv_files = glob.glob(os.path.join("../../custom-format-default-filtered-features/", "*.pickle"))

In [7]:
lst=[]
for f in tqdm(csv_files):
    df_tmp=pd.read_pickle(f)
    lst.append(df_tmp)
data_df=pd.concat(lst)

100%|██████████| 585/585 [01:10<00:00,  8.26it/s]


In [8]:
from datetime import datetime
df_counts = data_df.groupby(['infobox_key', 'property_name'],sort=False).count()["timestamp"]

In [9]:
lst=[]
start=0
for key,count in tqdm(df_counts.iteritems()):
    if count>=200:
        lst.append(data_df[start:start+count])
    start+=count
data_df_small=pd.concat(lst)
data_df_small.shape

6184429it [00:04, 1310077.80it/s]


(645280, 21)

In [10]:
#number of keys
data_df_small.groupby(['infobox_key', 'property_name'],sort=False).count()["timestamp"].shape

(2452,)

In [11]:
framework.data = data_df_small

In [12]:
framework.data["value_valid_from"] = pd.to_datetime(framework.data["timestamp"]).dt.tz_localize(None)

In [13]:
group_key=['infobox_key', 'property_name']
framework.data["key"] = list(
            zip(*[framework.data[group_key] for group_key in framework.group_key])
        )

In [14]:
framework.fit_model()

Start training.


  0%|          | 0/2445 [00:00<?, ?it/s]

Finished training. Time elapsed: 0:01:14.733942


In [15]:
# print(framework.test_model(randomize=False, predict_subset=1)

In [16]:
# framework.generate_plots()

In [18]:
import numpy as np
thresholds = np.linspace(0,1,11)
stats={"daily":{"prec":[],"rec":[]},
      "weekly":{"prec":[],"rec":[]},
      "monthly":{"prec":[],"rec":[]},
      "yearly":{"prec":[],"rec":[]},}
for threshold in thresholds:
    framework.predictor.threshold=threshold
    framework.test_model(randomize=False, predict_subset=1,save_results=False)
    for key,arr in zip(stats.keys(),framework.pred_stats):
        stats[key]["prec"].append(arr[0][1])
        stats[key]["rec"].append(arr[1][1])
        print( stats[key]["prec"], stats[key]["rec"])

  0%|          | 0/2452 [00:00<?, ?it/s]

Starting evaluation.
Finished evaluation. Time elapsed: 0:00:00.645031


AttributeError: 'TrainAndPredictFramework' object has no attribute 'pred_stats'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
key="daily"
fig=sns.lineplot(data=pd.DataFrame(stats[key],thresholds))
fig.set_title(key)
fig.set_xlabel("threshold")
plt.savefig(key+'.png')

In [ ]:
import seaborn as sns
key="weekly"
fig=sns.lineplot(data=pd.DataFrame(stats[key],thresholds))
fig.set_title(key)
fig.set_xlabel("threshold")
plt.savefig(key+'.png')

In [ ]:
import seaborn as sns
key="monthly"
fig=sns.lineplot(data=pd.DataFrame(stats[key],thresholds))
fig.set_title(key)
fig.set_xlabel("threshold")
plt.savefig(key+'.png')

In [ ]:
import seaborn as sns
key="yearly"
fig=sns.lineplot(data=pd.DataFrame(stats[key],thresholds))
fig.set_title(key)
fig.set_xlabel("threshold")
plt.savefig(key+'.png')

In [ ]:
stats